In [60]:
import pandas as pd

pd.set_option('display.max_rows', 500)


In [61]:
# Import FRED look up csv
fred = pd.read_csv(
    '~/Downloads/FRED2_csv_2/README_TITLE_SORT.txt', 
    skiprows=9, 
    skipfooter=22, 
    sep=';',
    parse_dates=[5],
    skipinitialspace=True,
    engine='python'
)
fred.columns = fred.columns.str.strip()
# Remove all "(DISCONTINUED)" rows
fred = fred[~fred['Title'].str.contains('DISCONTINUED')]
fred = fred.set_index('Title')


In [62]:
# Get all U.S. population data
uspop = fred[fred.index.str.match('Population.*United States.*')]
# Remove % change units
uspop = uspop[~uspop['Units'].str.contains('%')]
# Find the latest
uspop = uspop.loc[uspop['Last Updated'].argmax()].iloc[0]
uspop

File                   P\O\P\T\O\POPTOTUSA647NWDB.csv          
Units                                                   Persons
Frequency                                                     A
Seasonal Adjustment                                         NSA
Last Updated                                2015-07-07 00:00:00
Name: Population, Total for United States, dtype: object

In [63]:
# Get all personal income/exspense data
personal = fred[fred.index.str.match('Personal.*')]

# Remove State based data
personal = personal[~personal.index.str.contains('.*, [A-Z]{2}')]
personal = personal[~personal.index.str.contains('.*for Social Insurance in.*')]
personal = personal[~personal.index.str.contains('.*District of Columbia.*')]

# Remove per capitas
personal = personal[~personal.index.str.contains('.*per capita.*')]

# Remove old 2012 style data
personal = personal[~personal.index.str.contains('.*,.* for United States')]

# Remove nonresidents
personal = personal[~personal.index.str.contains('.*nonresidents.*')]

# Remove % change and index units
personal = personal[~personal['Units'].str.contains('%|Index|Persons|National Currency')]

# Get only monthly, quartarly or yearly data
personal = personal[personal['Frequency'].str.contains('M|Q|A')]

# Remap frequency to number of months
personal['Frequency'].replace({'M':1, 'Q':3, 'A':12}, inplace=True)

# Keep only the highest resolution of duplicate data
personal = personal.reset_index() \
    .sort_values(['Title', 'Frequency', 'Last Updated'], ascending=[1, 1, 0]) \
    .drop_duplicates(subset='Title', keep='first') \
    .sort_values('Title').set_index('Title')


In [64]:
# Convert to hierarchal index by spliting on ':'
personal.index = pd.MultiIndex.from_arrays(
    pd.DataFrame.from_dict({
        k: v for k, v in enumerate(personal.index.str.split(':'))
    }, orient='index').T.values
)

In [66]:
personal

File  \
Personal Consumption Expenditures                  NaN                                                NaN                                                NaN                                               NaN                    NaN                  P\C\PCE.csv                                
Personal Consumption Expenditures by Type of Pr...  Services                                           Household Consumption Expenditures                 Housing                                          NaN                    NaN                  D\H\DHSGRC0.csv                            
Personal Consumption Expenditures                   Durable Goods                                     NaN                                                NaN                                               NaN                    NaN                  P\C\E\PCEDG.csv                            
                                                    Goods                                             NaN                                                NaN                                               NaN                    NaN                  D\G\DGDSRC1.csv                            
                                                    Nondurable Goods                                  NaN                                                NaN                                               NaN                    NaN                  P\C\E\PCEND.csv                            
                                                    Services                                          NaN                                                NaN                                               NaN                    NaN                  P\C\E\PCES.csv                             
Personal Current Tax Receipts                       State and Local Government                         Property Taxes                                    NaN                                               NaN                    NaN                  S\2\S210400.csv                            
Personal Current Transfer Receipts                 NaN                                                NaN                                                NaN                                               NaN                    NaN                  P\C\T\PCTR.csv                             
Personal Income                                    NaN                                                NaN                                                NaN                                               NaN                    NaN                  P\PI.csv                                   
Personal Income Receipts on Assets                 NaN                                                NaN                                                NaN                                               NaN                    NaN                  P\I\R\PIROA.csv                            
                                                    Personal Dividend Income                          NaN                                                NaN                                               NaN                    NaN                  P\D\PDI.csv                                
                                                    Personal Interest Income                          NaN                                                NaN                                               NaN                    NaN                  P\I\PII.csv                                
Personal Income for United States                  NaN                                                NaN                                                NaN                                               NaN                    NaN                  M\0\8\M0868CUSM027SNBR.csv                 
Personal Saving                                    NaN                                                NaN                                                NaN                                               NaN   